
# Faces recognition example using eigenfaces and SVMs


The dataset used in this example is a preprocessed excerpt of the
"Labeled Faces in the Wild", aka LFW_:

  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)

.. _LFW: http://vis-www.cs.umass.edu/lfw/

Expected results for the top 5 most represented people in the dataset:

================== ============ ======= ========== =======
                   precision    recall  f1-score   support
================== ============ ======= ========== =======
     Ariel Sharon       0.67      0.92      0.77        13
     Colin Powell       0.75      0.78      0.76        60
  Donald Rumsfeld       0.78      0.67      0.72        27
    George W Bush       0.86      0.86      0.86       146
Gerhard Schroeder       0.76      0.76      0.76        25
      Hugo Chavez       0.67      0.67      0.67        15
       Tony Blair       0.81      0.69      0.75        36

      avg / total       0.80      0.80      0.80       322
================== ============ ======= ========== =======



In [1]:
from __future__ import print_function

from time import time
import logging
import matplotlib.pyplot as plt
import scipy

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC

#from sklearn.cross_validation import KFold
import numpy as np

print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

Automatically created module for IPython interactive environment


In [2]:
def samplingNum(start=1, coef=[5,2], size=1):
    list = []
    for i in range (size):
        list.append(start)
        start = start*coef[i%2]
    return list

Download the data, if not already on disk and load it as numpy arrays


In [3]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
#X_train = np.load('X_train.npy')
#y_train = np.load('y_train.npy')
#X_test = np.load('X_test.npy')

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
#n_features = X_train.shape[1]
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
print(target_names)
n_classes = target_names.shape[0]
#n_classes = y_train.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)
print('height: ', h)
print('width: ', w)
#print('X_train: ', X_train)
#print('y_train: ', y_train)

['Ariel Sharon' 'Colin Powell' 'Donald Rumsfeld' 'George W Bush'
 'Gerhard Schroeder' 'Hugo Chavez' 'Tony Blair']
Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7
height:  50
width:  37


Split into a training set and a test set using a stratified k fold


In [4]:
# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
dataset): unsupervised feature extraction / dimensionality reduction


In [5]:
n_components = 150
#h = 50
#w = 37

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))

Extracting the top 150 eigenfaces from 966 faces
done in 0.353s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.036s


Train a SVM classification model


In [6]:
C = samplingNum(start=0.1, size= 14)
gamma = samplingNum(start=0.0001,size= 10)
print(C)
print(gamma)

[0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0, 500.0, 1000.0, 5000.0, 10000.0, 50000.0, 100000.0, 500000.0]
[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0]


In [14]:
print("Fitting the classifier to the training set")
t0 = time()
#C[0.1,1,10,100,1e3, 5e3, 1e4, 5e4, 1e5]
#gamma[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05,0.1,0.5]
param_grid = [
{'C': [0.1,1,10,100,1e3, 5e3, 1e4, 5e4, 1e5], 'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05,0.1,0.5], 'kernel': ['rbf']}]

#param_grid = {'C': scipy.stats.expon(scale=100), 'gamma': scipy.stats.expon(scale=.1),
 # 'kernel': ['rbf'], 'class_weight':['balanced', None]}
clf = GridSearchCV(SVC(class_weight='balanced'), param_grid)

#clf = RandomizedSearchCV(SVC(class_weight='balanced'), param_grid)
#clf = SVC(kernel='rbf', C=10, gamma=0.005) -> f1 0.86
clf = clf.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")

fo = open('y_test_2.txt', 'w')
fo.write(str(clf.best_estimator_) + "\n")
fo.close()
print(clf.best_estimator_)
print(f1_score(y_test, clf.predict(X_test_pca), average='micro')) #for quick test

Fitting the classifier to the training set
done in 89.418s
Best estimator found by grid search:
SVC(C=10, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.844720496894


Quantitative evaluation of the model quality on the test set


In [ ]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))
#print(y_pred)

print(f1_score(y_test, y_pred, average='micro'))
print(classification_report(y_test, y_pred, target_names=target_names))
#print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))
fo = open('out.txt', 'w')
fo.write("ImageId,PredictedClass"+"\n")
for i in range(y_pred.size):
    fo.write(str(i) + "," + str(y_pred[i]) + "\n")
    print(str(i) + "," + str(y_pred[i]))
fo.close()

Qualitative evaluation of the predictions using matplotlib


In [ ]:
#def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
     #Helper function to plot a gallery of portraits
#    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
#    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
#    for i in range(n_row * n_col):
#        plt.subplot(n_row, n_col, i + 1)
#        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
#        plt.title(titles[i], size=12)
#        plt.xticks(())
#        plt.yticks(())


# plot the result of the prediction on a portion of the test set

#def title(y_pred, y_test, target_names, i):
#    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
#    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
#    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

#prediction_titles = [title(y_pred, y_test, target_names, i)
#                     for i in range(y_pred.shape[0])]

#plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

#eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
#plot_gallery(eigenfaces, eigenface_titles, h, w)

#plt.show()